In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf

VBox()

Starting Spark application


In [ ]:
spark = SparkSession \
    .builder \
    .appName("Spark Vocabulary statistics") \
    .getOrCreate()

In [ ]:
data_matched_path = "s3://multinli/MNLI/dev_matched.tsv"
data_test_path = "s3://multinli/MNLI/test_matched.tsv"

data_mismatch_path = "s3://multinli/MNLI/dev_mismatched.tsv"
data_test_mismatch_path = "s3://multinli/MNLI/test_mismatched.tsv"

stoplist = "s3://multinli/utils/stopwords.txt"

# path
# s3://multinli/MNLI/dev_matched.tsv
# s3://multinli/MNLI/test_matched.tsv  
# s3://multinli/MNLI/dev_mismatched.tsv
# s3://multinli/MNLI/test_mismatched.tsv

cols=['genre','sentence1','sentence2']


def read_formate(path):
    rawData = spark.read.csv(path,header=True,sep='\t',mode='DROPMALFORMED')
    
    return rawData.select(cols).cache()

# data_matched = spark.read.csv(data_matched_path,header=True,sep='\t',mode='DROPMALFORMED')

# data_test_matched = spark.read.csv(data_test_path,header=True,sep='\t',mode='DROPMALFORMED')


getData = read_formate(data_matched_path)
getTestData = read_formate(data_test_path)

getMismatchDev = read_formate(data_mismatch_path)
getMismatchText = read_formate(data_test_mismatch_path)


In [ ]:
# getData.show(5)

In [ ]:
# concatSentence = getData.withColumn('joined_sentence', sf.concat(sf.col('sentence1'),sf.lit(' '),\
#                                                         sf.col('sentence2'))).cache()

def concat2Senten(record):
    '''
    concat sentence1 and sentence2 together
    
    '''
    return record.withColumn('joined_sentence', sf.concat(sf.col('sentence1'),sf.lit(' '),\
                                                        sf.col('sentence2')))
def wordTokenize(record):
    '''
    lower each letter and word_tokenize
    '''
    try:
        gerne,senten1, senten2,joined_senten = record
        lowerW = joined_senten.lower()
        
        tokens = word_tokenize(lowerW)
        # remove all tokens that are not alphabetic
        words = [word for word in tokens if word.isalpha()]

        return words
    except:
        print(joined_senten)
        return ()


In [ ]:
# concatSentence.show(5)

In [ ]:

from nltk.tokenize import word_tokenize
# from nltk.tokenize.stanford import StanfordTokenizer as tk
# tk = StanfordTokenizer() 

concatSentence = concat2Senten(getData)
concatSentenceTest = concat2Senten(getTestData)

# union dev and test set together
uniMatch = concatSentence.union(concatSentenceTest)

words = uniMatch.rdd.flatMap(wordTokenize)


In [15]:
# uniMatch.count()
concatSentence.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+--------------------+
|     genre|           sentence1|           sentence2|     joined_sentence|
+----------+--------------------+--------------------+--------------------+
|     slate|The new rights ar...|Everyone really l...|The new rights ar...|
|government|This site include...|The Government Ex...|This site include...|
| telephone|uh i don't know i...|I like him for th...|uh i don't know i...|
| telephone|yeah i i think my...|My favorite resta...|yeah i i think my...|
| telephone|i don't know um d...|     I know exactly.|i don't know um d...|
+----------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [17]:
# words.take(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
'''
1. stopWords.value read Stanford stop list from 
2. getValues stopList to list
3. remove some stopwords
'''

stopWords = spark.read.text(stoplist)
stopList = stopWords.rdd.map(lambda e: e.value).collect()
#remove some stop words
stopW = words.filter(lambda word : word not in stopList and word[0] != '')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# print(stopW.collect()) 

# #remove some stop words
# stopW = words.filter(lambda word : word[0] not in stop_words and word[0] != '')
# #remove punctuation in sentence
# filtered_data = stopW.filter(lambda punct : punct not in list_punct)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
countWord = stopW.map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) # count each words
# wordSorted = countWord.sortBy(lambda x: x[1], ascending=False).cache() #sort by counted value 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# wordSorted.take(5) #get top 5  common words 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'wordSorted' is not defined
Traceback (most recent call last):
NameError: name 'wordSorted' is not defined



## Mistmatch word

In [23]:
concatSentence = concat2Senten(getMismatchDev).cache()
concatSentenceTest = concat2Senten(getMismatchText).cache()
# union dev and test set together
uniMismatch = concatSentence.union(concatSentenceTest)

# uniMismatch.count()

words = uniMismatch.rdd.flatMap(wordTokenize)

# stopWords = spark.read.text(stoplist)
# stopList = stopWords.rdd.map(lambda e: e.value).collect()
#remove some stop words
stopW = words.filter(lambda word : word not in stopList and word[0] != '')


mismatchWord = stopW.map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) # count each words
# mismatchWord = mismatchWord.sortBy(lambda x: x[1], ascending=False)#sort by counted value 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# mismatchWord.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
allDataWords = countWord.union(mismatchWord)


from operator import add

commonWords = allDataWords.reduceByKey(add).sortBy(lambda x: x[1], ascending=False).collect()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
print(commonWords[0:10])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('one', 3593), ('will', 3159), ('like', 2996), ('know', 2916), ('uh', 2406), ('time', 2368), ('people', 2241), ('just', 2004), ('new', 1937), ('well', 1780)]

In [27]:
matchedUni = countWord.filter(lambda x: x[1]==1).count()

matchedUni

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2454

In [28]:
mismatchedUni = mismatchWord.filter(lambda x: x[1]==1).count()

mismatchedUni

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2294

In [29]:
test = countWord.filter(lambda x: x[1]==1)

test.takeSample(False, 10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('erosion', 1), ('optimally', 1), ('shrieked', 1), ('dies', 1), ('powerless', 1), ('alerting', 1), ('wwii', 1), ('cheerfully', 1), ('farmed', 1), ('paternal', 1)]

## Taining data set  Component Analysis


In [1]:
from pyspark.sql import SparkSession
from nltk.tokenize import word_tokenize
from pyspark.sql import functions as sf

ss = SparkSession \
    .builder \
    .appName("Taining Dataset Component Analysis") \
    .getOrCreate()

stoplist = "s3://multinli/utils/stopwords.txt"
stopWords = ss.read.text(stoplist)
stopList = stopWords.rdd.map(lambda e: e.value).collect()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
13,application_1589244391897_0014,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:

def concat2Senten1(record):
    '''
    concat sentence1 and sentence2 together
    
    '''   
    return  record.withColumn('joined_sentence', sf.concat(sf.col('sentence1'),sf.lit(' '),\
                                                        sf.col('sentence2')))

def read_formate(path):
    rawData = ss.read.csv(path,header=True,sep='\t',mode='DROPMALFORMED')
    
    return rawData.select(cols)

def wordTokenize1(record):
    '''
    lower each letter and word_tokenize
    '''
    try:
        gerne,joined_senten = record
        
        lowerW = joined_senten.lower()
        
        tokens = word_tokenize(lowerW)
        # remove all tokens that are not alphabetic
        words = [word for word in tokens if word.isalpha()]
        
#         words = [word for word in tokens if word not in stopList and word.isalpha()] 
        
#         words = [word for word in delteNotAlpha if  word not in stopList]
            
#         return (gerne,words)
        return words
    
    except:
        print(joined_senten)
        return ()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
data_train_path = "s3://multinli/MNLI/train.tsv"

cols=['genre','sentence1','sentence2']

readTrainData = read_formate(data_train_path)

trian_data = concat2Senten1(readTrainData)

# #remove some stop words
# reStopWord = trainWords.filter(lambda word : word[1] not in stopList and word[0] != '')

# mismatchWord = reStopWord.map(lambda word: (word, 1)) \
#     .reduceByKey(lambda a, b: a + b) # count each words

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
selectData = trian_data.select(['genre','joined_sentence'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# trainWords = trian_data.rdd.flatMap(wordTokenize1).cache()

reStopWord = selectData.rdd.flatMap(wordTokenize1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# reStopWord.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['conceptually', 'cream', 'skimming', 'has', 'two']

In [6]:

stopW = reStopWord.filter(lambda word : word not in stopList and word[0] != '')


test = stopW.map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], ascending = False).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
 test.take(lambda x: x[1] != 1)

trainUniWord = test.filter(lambda x: x[1]==1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
trainUniWord.take(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('recharged', 1), ('hosue', 1), ('japaneese', 1), ('incabable', 1), ('baeuerstein', 1), ('posessing', 1), ('stoicism', 1), ('purging', 1), ('layabouts', 1), ('encroachment', 1), ('kenya', 1), ('exteremly', 1), ('directino', 1), ('vicotry', 1), ('perretti', 1), ('contravene', 1), ('exonerates', 1), ('neurolinguistics', 1), ('meteorites', 1), ('orignal', 1), ('traditioal', 1), ('minnesora', 1), ('comples', 1), ('metabolized', 1), ('aera', 1), ('sauté', 1), ('yesteray', 1), ('dystopian', 1), ('palacia', 1), ('airshow', 1), ('capitvate', 1), ('kardemana', 1), ('vesuvious', 1), ('repoed', 1), ('strum', 1), ('lilting', 1), ('amphitre', 1), ('inprisonment', 1), ('indianpolis', 1), ('alloewd', 1), ('comjes', 1), ('biofuel', 1), ('happes', 1), ('earful', 1), ('techiques', 1), ('hgih', 1), ('thistorical', 1), ('meetinfs', 1), ('overs', 1), ('leashes', 1), ('aghisasos', 1), ('gradeur', 1), ('atmospheres', 1), ('phine', 1), ('mariticide', 1), ('penisula', 1), ('woulf', 1), ('jerusalemites', 1), (